Help on class SWMWriter in module WeightsUtilities:

class SWMWriter(builtins.object)
 |  SWMWriter(swmFile, masterField, spatialRefName, numObs, rowStandard, inputFC='#', wType=-1, distanceMethod='#', exponent='#', threshold='#', numNeighs='#', inputTable='#', timeField='#', timeType='#', timeValue='#', inputNet='#', impedanceField='#', barrierFC='#', uturnPolicy='#', restrictions='#', useHierarchy='#', searchTolerance='#', addConcept='#', forceFixed=False, hasZ=False)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, swmFile, masterField, spatialRefName, numObs, rowStandard, inputFC='#', wType=-1, distanceMethod='#', exponent='#', threshold='#', numNeighs='#', inputTable='#', timeField='#', timeType='#', timeValue='#', inputNet='#', impedanceField='#', barrierFC='#', uturnPolicy='#', restrictions='#', useHierarchy='#', searchTolerance='#', addConcept='#', forceFixed=False, hasZ=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  close(self)
 | 

In [2]:
from pysal.lib import weights
import geopandas as gpd
import pandas as pd
import WeightsUtilities as WU
import numpy as np
from scipy.spatial.distance import cdist
from arcpy.stats import GenerateSpatialWeightsMatrix

c:\Users\Lenovo\.conda\envs\arcgispro-spa-env\lib\site-packages\pysal\explore\segregation\network\network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
c:\Users\Lenovo\.conda\envs\arcgispro-spa-env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [4]:
shp_path="F:\大创数据\中间产出的数据\云南省和黄淮海平原已处理好的火点\黄淮海平原逐月火点\黄淮海平原已处理好的火点_1月.shp"
swm_path="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_1月.swm"
gdf = gpd.read_file(shp_path)

# 提取所需的字段（ID, X, Y, Z）
df = gdf[['ID', 'X', 'Y', 'Z']].copy()
coordinates = gdf[['X', 'Y', 'Z']].values

master_field="ID"
spatial_ref_name="World_Mercator"
# 设置距离阈值L0
L0 = 55000

# 计算空间权重矩阵
    #计算要素间的距离
    # coordinates = df[['X', 'Y']].values
distances = cdist(coordinates, coordinates)

    # 对得到的距离进行判断并赋值
spatial_weights = np.where((distances<=L0)&(distances!=0), 1, 0)

# 将空间权重矩阵保存到DataFrame中
spatial_weights_df = pd.DataFrame(spatial_weights)


# DataFrame转list转换函数
def df_to_list(df):
    result = []
    for idx, row in df.iterrows():
        element = idx
        neighbors = []
        weights = []
        for col, val in row.iteritems():
            if val != 0:
                neighbors.append(col)
                weights.append(val)
        result.append((element, neighbors, weights))
    return result

spatial_weights_list=df_to_list(spatial_weights_df)

#调用SWMWriter将spatial_weights_list写入swm文件
#格式如下：swm1.swm.writeEntry(要素,[邻居要素1,2,3……], [邻居要素的权重1,2,3……])
swm1=WU.SWMWriter(swm_path,master_field,spatial_ref_name,len(df),rowStandard=True)
for i in spatial_weights_list:
    try: 
        if i[1] is None:
            swm1.swm.writeEntry(i[0],[],[])
        else:
            swm1.swm.writeEntry(i[0],i[1],i[2])
    except:
        print("---------",i,"---------")

swm1.close()